In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error, mean_absolute_error,r2_score,mean_squared_error,accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, train_test_split
#from sklearn.linear_model import LinearRegression,RANSACRegressor,Lasso,BayesianRidge,ElasticNet
import matplotlib.pyplot as plt
import seaborn as sns
#import dtale
#import dtale.app as dtale_app
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.svm import SVR
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier#Bagging
from sklearn.ensemble import BaggingClassifier #Bagging
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
#from pandas_profiling import ProfileReport
from xgboost import XGBClassifier
from sklearn.tree import _tree
%matplotlib inline

In [3]:
ids_datatypes = {
    'Dst Port': np.int32,
    'Protocol': np.int8,
    'Flow Duration': np.int64,
    'Tot Fwd Pkts': np.int16,
    'Tot Bwd Pkts': np.int16,
    'TotLen Fwd Pkts': np.int32,
    'TotLen Bwd Pkts': np.int32,
    'Fwd Pkt Len Max': np.int32,
    'Fwd Pkt Len Min': np.int32,
    'Fwd Pkt Len Mean': np.float64,
    'Fwd Pkt Len Std': np.float64,
    'Bwd Pkt Len Max': np.int16,
    'Bwd Pkt Len Min': np.int16,
    'Bwd Pkt Len Mean': np.float64,
    'Bwd Pkt Len Std': np.float64,
    'Flow Byts/s': np.float64,
    'Flow Pkts/s': np.float64,
    'Flow IAT Mean': np.float64,
    'Flow IAT Std': np.float64,
    'Flow IAT Max': np.int64,
    'Flow IAT Min': np.int32,
    'Fwd IAT Tot': np.int32,
    'Fwd IAT Mean': np.float32,
    'Fwd IAT Std': np.float64,
    'Fwd IAT Max': np.int32, 
    'Fwd IAT Min': np.int32,
    'Bwd IAT Tot': np.int32,
    'Bwd IAT Mean': np.float64,
    'Bwd IAT Std': np.float64,
    'Bwd IAT Max': np.int64,
    'Bwd IAT Min': np.int64,
    'Fwd PSH Flags': np.int8,
    'Bwd PSH Flags': np.int8,
    'Fwd URG Flags': np.int8,
    'Bwd URG Flags': np.int8,
    'Fwd Header Len': np.int32,
    'Bwd Header Len': np.int32,
    'Fwd Pkts/s' : np.float64,
    'Bwd Pkts/s': np.float64,
    'Pkt Len Min': np.int16,
    'Pkt Len Max': np.int32,
    'Pkt Len Mean': np.float64,
    'Pkt Len Std': np.float64,
    'Pkt Len Var': np.float64,
    'FIN Flag Cnt': np.int8,
    'SYN Flag Cnt': np.int8,
    'RST Flag Cnt': np.int8,
    'PSH Flag Cnt': np.int8,
    'ACK Flag Cnt': np.int8,
    'URG Flag Cnt': np.int8,
    'CWE Flag Count': np.int8,
    'ECE Flag Cnt': np.int8,
    'Pkt Size Avg': np.float32,
    'Fwd Seg Size Avg': np.float32,
    'Bwd Seg Size Avg': np.float32,
    'Fwd Byts/b Avg': np.int8,
    'Fwd Pkts/b Avg': np.int8,
    'Fwd Blk Rate Avg': np.int8,
    'Bwd Byts/b Avg': np.int8,
    'Bwd Pkts/b Avg': np.int8,
    'Bwd Blk Rate Avg': np.int8,
    'Subflow Fwd Pkts': np.int16,
    'Subflow Fwd Byts': np.int32,
    'Subflow Bwd Pkts': np.int16,
    'Subflow Bwd Byts': np.int32,
    'Init Fwd Win Byts': np.int32, 
    'Init Bwd Win Byts': np.int32,
    'Fwd Act Data Pkts': np.int16,
    'Fwd Seg Size Min': np.int8,
    'Active Mean': np.float64,
    'Active Std': np.float64,
    'Active Max': np.int32,
    'Active Min': np.int32,
    'Idle Mean': np.float64,
    'Idle Std': np.float64,
    'Idle Max': np.int64,
    'Idle Min': np.int64,
    'Label': object
}
used_cols = (ids_datatypes.keys())

In [4]:
df1 = pd.read_csv("../input/ids-intrusion-csv/02-14-2018.csv", dtype=ids_datatypes, usecols=used_cols)
df2 = pd.read_csv("../input/ids-intrusion-csv/02-15-2018.csv", dtype=ids_datatypes, usecols=used_cols)
#df3 = pd.read_csv("../input/ids-intrusion-csv/02-16-2018.csv", dtype=ids_datatypes, usecols=used_cols)
#df4 = pd.read_csv("../input/ids-intrusion-csv/02-20-2018.csv", dtype=ids_datatypes, usecols=used_cols)
df5 = pd.read_csv("../input/ids-intrusion-csv/02-21-2018.csv", dtype=ids_datatypes, usecols=used_cols)
df6 = pd.read_csv("../input/ids-intrusion-csv/02-22-2018.csv", dtype=ids_datatypes, usecols=used_cols)
df7 = pd.read_csv("../input/ids-intrusion-csv/02-23-2018.csv", dtype=ids_datatypes, usecols=used_cols)
#df8 = pd.read_csv("../input/ids-intrusion-csv/02-28-2018.csv", dtype=ids_datatypes, usecols=used_cols)
#df9 = pd.read_csv("../input/ids-intrusion-csv/03-01-2018.csv", dtype=ids_datatypes, usecols=used_cols)
df10 = pd.read_csv("../input/ids-intrusion-csv/03-02-2018.csv", dtype=ids_datatypes, usecols=used_cols)

In [5]:
merge = [
    df1, 
    df2,
    df5,
    df6, 
    df7, 
    df10
]
df_ids = pd.concat(merge)
del df1
del df2
del df5
del df6
del df7
del df10
del merge
#df_ids.info()

In [6]:
label_dict = {
    'Benign': 1,
    'FTP-BruteForce': 2,
    'SSH-Bruteforce': 3,
    'DDOS attack-HOIC': 4,
    'Bot': 5,
    'DoS attacks-GoldenEye' : 6,
    'DoS attacks-Slowloris': 7,
    'DDOS attack-LOIC-UDP': 8,
    'Brute Force -Web': 9,
    'Brute Force -XSS': 10,
    'SQL Injection': 11
}
df_ids['Label'] = [label_dict[item] for item in df_ids['Label']]
df_ids = df_ids.replace([np.inf, -np.inf], np.nan)

In [7]:
label_undict = {
    1:'Benign',
    2:'FTP-BruteForce',
     3 :'SSH-Bruteforce',
    4 :'DDOS attack-HOIC',
     5:'Bot',
     6:'DoS attacks-GoldenEye' ,
     7:'DoS attacks-Slowloris',
     8:'DDOS attack-LOIC-UDP',
     9:'Brute Force -Web',
     10:'Brute Force -XSS',
     11:'SQL Injection'
}

In [8]:
def cleandata(df,ids="",target=""):
    scale = MinMaxScaler()#scaler
    #print(df)
    cols=df.columns#name of columns
    for label, content in df.items():#identifying category type features
        if pd.api.types.is_string_dtype(content):
            df[label] = content.astype("category").cat.as_ordered()

    for label, content in df.items():#handling missing data in Numeric type
        if pd.api.types.is_numeric_dtype(content):
             if pd.isnull(content).sum():
                    # Fill missing numeric values with median since it's more robust than the mean
                    #print(label)
                    df[label] = content.fillna(content.median())  
    rev=[]
    Encoder=LabelEncoder()
    for label, content in df.items():
        # Check columns which *aren't* numeric
         if not pd.api.types.is_numeric_dtype(content):
                #encoding category type features into numeric format
                rev.append(label)
                df[label]=Encoder.fit_transform(df[label])
    scaling=[]
    for label,content in df.items():#scaling the data excluding Target
        if (label!=ids and label not in rev ):
            scaling.append(label)
    if(len(target)>0):scaling.remove(target)
    df[scaling]=scale.fit_transform(df[scaling])
    #df[label]=scaled(df[label])
    #print(df)
    return df

for label, content in df_ids.items():#handling missing data in Numeric type
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # Fill missing numeric values with median since it's more robust than the mean
            #print(label)
            df_ids[label] = content.fillna(content.median())


In [9]:
y = df_ids[['Label']]
X = df_ids.drop('Label',axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.1)
del df_ids

In [ ]:
def model_selection(X_train, X_test, y_train, y_test):#function to select optimum model
    mod = {   "Bernouli":BernoulliNB(),
              "Perceptron":Perceptron(),
              #"KNC" :KNeighborsClassifier(n_neighbors=11),
              "Gausian":GaussianNB(),
              "DTree":DecisionTreeClassifier(max_depth=15),
              "Rforest":RandomForestClassifier(max_depth=15),
              "ETree":ExtraTreesClassifier(max_depth=15),
              #"Gradient":GradientBoostingClassifier(n_estimators=500, max_depth=10, learning_rate=0.01),
              "Ada":AdaBoostClassifier(),
              #"XGB":XGBClassifier(),
              "Logistic":LogisticRegression(max_iter=150)
              }
    Err={}
    for key,value in mod.items():#mapping models with their accuracy score
        mod[key].fit(X_train,y_train)
        pred_test=mod[key].predict(X_test)
        Err[key]=accuracy_score(y_test,pred_test)
        print(f"{key} : {Err[key]}")

    plt.xticks(rotation = 45) #ploting models with thier respective accuracy score
    plt.bar(Err.keys(),Err.values(),align='center')
    plt.show()
model_selection(X_train, X_test, y_train, y_test)

In [11]:
Dtree = DecisionTreeClassifier(max_depth=15, random_state=1234)
model = Dtree.fit(X, y)

In [12]:
f = open("output.txt", "a")
def tree_to_code(tree, feature_names):
    
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    feature_names = [f.replace(" ", "_")[:-5] for f in feature_names]
    print("def predict({}):".format(", ".join(feature_names)),file=f)

    def recurse(node, depth):
        indent = "    " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, np.round(threshold,2)),file=f)
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, np.round(threshold,2)),file=f)
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree_.value[node]),file=f)

    recurse(0, 1)
tree_to_code(model,list(ids_datatypes.keys()))
f.close()

In [13]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules
rule=get_rules(model,list(ids_datatypes.keys()) , list(label_dict.keys()))
f = open("output_human.txt", "a")
print(rule,file=f)
f.close()

In [14]:
import pickle
with open('Dtree.pkl', 'wb') as f:
    pickle.dump(model, f)
#with open('Dtree.pkl', 'rb') as f:
    #clf = pickle.load(f)